In [3]:
#Je achete le boeuf!
#plotting imports
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# ignore warnings
import warnings
warnings.filterwarnings("ignore")


#modeling imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#pipeline modules
import acquire
import explore
import prepare

In [4]:
df = prepare.prep_titanic()

In [6]:
df.dropna(inplace=True)
df.isnull().sum()

passenger_id    0
survived        0
pclass          0
sex             0
age             0
sibsp           0
parch           0
fare            0
embarked        0
class           0
embark_town     0
alone           0
dtype: int64

In [8]:
#DATA IS ACQUIRED AND PREPARED.
#LETS DO THE 4 WAY SPLIT OF X-y, and each of those are split into train and test.
X=df[['pclass','age','fare','sibsp','parch']]
y=df[['survived']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.30,random_state=123)

,pclass,age,fare,sibsp,parch
60,3,22.0,0.014110,0,0
348,3,3.0,0.031035,1,1
606,3,30.0,0.015412,0,0
195,1,58.0,0.285990,0,0
56,2,21.0,0.020495,0,0


In [9]:
logit = LogisticRegression(C=1,class_weight={1:2},random_state=123,solver='saga')

In [10]:
logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
x = logit.coef_

In [18]:
print(f"The coefficients for the predictors are {x}.")
print(f"The y-intercept for the model is {logit.intercept_}.")

The coefficients for the predictors are [[-0.42889369  0.00886309  0.25218707 -0.07228062  0.53071595]].
The y-intercept for the model is [0.69233036].


In [19]:
#An array of predicted labels of the target variable generated by fitting the Xtrain and ytrain to LogReg model
#Some are right, some are wrong.
y_pred = logit.predict(X_train)

In [20]:
y_pred_proba = logit.predict_proba(X_train)

In [23]:
blob = confusion_matrix(y_train, y_pred)

In [24]:
#That little insignificant 2x2 matrix is the confusion matrix. 
print(blob)

[[169 124]
 [ 52 154]]


# Looks like terrible precision but okay recall.

In [30]:
cr=(classification_report(y_train,y_pred))

In [33]:
score = logit.score(X_test,y_test)

# Why do all that work when sklearn.metrics.classification_report will do it for us

In [31]:
print(cr)

              precision    recall  f1-score   support

           0       0.76      0.58      0.66       293
           1       0.55      0.75      0.64       206

    accuracy                           0.65       499
   macro avg       0.66      0.66      0.65       499
weighted avg       0.68      0.65      0.65       499



In [39]:
true_negative = blob[0][0]
true_positive = blob[1][1]
false_negative = blob[1][0]
false_positive = blob[0][1]

In [40]:
accuracy=((true_negative+true_positive)/len(y_train))
#all the correctly predicted positives divided by that AND the ones that were predicted negative but shouldn't have
recall=true_positive/(true_positive+false_negative)
#All the corretly predicted positives, divied by that and the ones that the model was too eager to predict as positive
precision=true_positive/(true_positive+false_positive)
#Works horizontally along the actual positive. FP/FP+TN
false_positive_rate=false_positive/(false_positive+true_negative)

In [42]:
print(f"The accuracy is {accuracy}")
print(f"The recall is {recall}")
print(f"The precision is {precision}")

The accuracy is 0.6472945891783567
The recall is 0.7475728155339806
The precision is 0.5539568345323741


In [43]:
def do_the_logRegression(X_train, y_train, my_solver):
    logit = LogisticRegression(C=1,class_weight={1:2},random_state=123,solver=my_solver)
    logit.fit(X_train, y_train)
    y_pred = logit.predict(X_train)
    y_pred_proba=logit.predict_proba(X_train)
    score = logit.score(X_train,y_train)
    return score

In [44]:
do_the_logRegression(X_train, y_train, 'lbfgs')

0.7014028056112225

In [47]:
solvers = ["lbfgs", "liblinear", "sag", "saga", "newton-cg"]

In [50]:
for i in solvers:
    print(f"The accuracy for {i} as the solver is {do_the_logRegression(X_train, y_train, i)}")

The accuracy for lbfgs as the solver is 0.7014028056112225
The accuracy for liblinear as the solver is 0.7014028056112225
The accuracy for sag as the solver is 0.657314629258517
The accuracy for saga as the solver is 0.6472945891783567
The accuracy for newton-cg as the solver is 0.7014028056112225
